# Projeto 1 - Ciência dos Dados

Nome: Duilio Caoam Beneducci Moraes

Nome: Victoria Leal Garcia de Souza

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'dados.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo dados.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,"rt @dudacoutin_: vontade de comer brigadeiro, ...",1
1,qm q vai faze um brigadeiro branco p mim to co...,1
2,@vallnicolau tá agora eu to com vontade de com...,1
3,me deu uma vontade de comer brigadeiro com mor...,1
4,tava com vontade de comer abacaxi e brigadeiro...,1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,relevancia
0,meu deus ea vontade d um brigadeiro pra hj,1
1,"to morrendo de vontade de um brigadeiro, mas s...",0
2,vontade de comer brigadeiro quase queimado,1
3,uma vontade imensa de comer brigadeiro tô nem ...,1
4,a vontade de comer brigadeiro tá grande,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
quer=''
naoquer=''
for i in range(len(train.Relevancia)):
    if train.Relevancia[i] == 1:
        quer=quer+train.Treinamento[i]

for i in range(len(train.Relevancia)):
    if train.Relevancia[i] == 0:
        naoquer=naoquer+train.Treinamento[i]

In [6]:
import re 
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;,(\n)('')("")/@(rt)(\)]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
#limpa o texto
quer_limpo=cleanup(quer.lower())
naoquer_limpo=cleanup(naoquer.lower())

#espaçamento correto #não conseguimos ;-;😭
alfabeto = ['a','b', 'c', 'd', 'e', 'f', 'g', 'h', 'i' , 'j', 'k' , 'l', 'm' , 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
def limpa(x):
    lista=list(x)
    x=' '
    for i in range(len(lista)):
        if lista[i] not in alfabeto:
            lista[i]= ' ' + lista[i] + ' '
        elif lista[i] in alfabeto:
            lista[i]=lista[i]
    x =''.join(lista)
#tranforma em um conjunto de palavras
palavras_quer=quer_limpo.split()
palavras_naoquer=naoquer_limpo.split()

#gera uma planilha
serie_quer=pd.Series(palavras_quer)
serie_naoquer=pd.Series(palavras_naoquer)

#conta as quantidades
tabela_quer=serie_quer.value_counts()
tabela_naoquer=serie_naoquer.value_counts()

#faz as quantidades relativas
relativo_quer=serie_quer.value_counts(True)
relativo_naoquer=serie_naoquer.value_counts(True)

In [8]:
#definindo variaveis
probquer = len(serie_quer)/(len(serie_quer)+len(serie_naoquer))
probnaoquer = len(serie_naoquer)/(len(serie_quer)+len(serie_naoquer))

def classifica(x):
    x.insert(2,"treinado",'')
    for i in range(len(x.Teste)):
        x2=''
        x2=x.Teste[i]
    
        x2=cleanup(x2.lower())
        x3=x2.split()

        probFraseDadoq= 1
        probFraseDadon= 1

        for i in range(len(x3)):
            if x3[i] in relativo_quer:
                probFraseDadoq= probFraseDadoq *relativo_quer[x3[i]]
                if x3[i] in relativo_naoquer:
                    probFraseDadon=probFraseDadon *relativo_naoquer[x3[i]]

            elif x3[i] in relativo_naoquer:
                probFraseDadon= probFraseDadon *relativo_naoquer[x3[i]]
                if x3[i] in relativo_quer:
                    probFraseDadoq= probFraseDadoq *relativo_quer[x3[i]]

        probFraseDadoq=probFraseDadoq*probquer
        probFraseDadon=probFraseDadon*probnaoquer

        if (probFraseDadoq/probFraseDadon) > (probFraseDadon/probFraseDadoq) :
            x.treinado[i]=1
        elif (probFraseDadoq/probFraseDadon) < (probFraseDadon/probFraseDadoq) :
            x.treinado[i]=0
classifica(test)
test.treinado.value_counts()

C:\Users\duili\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\duili\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


     128
1     22
0      9
Name: treinado, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**